In [1]:
import sklearn.feature_selection
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectPercentile, chi2,RFE,f_classif, mutual_info_classif
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import SelectFromModel
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')
from statistics import mean
import time
from func_timeout import FunctionTimedOut, func_timeout
from sklearn.utils import resample
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import *
import statistics

# Import data

In [2]:

data = pd.read_csv('LICD.csv', sep=';')

new_columns = (data.columns.drop(['Industry Main Area Code','Fiscal accounting type','Currency code','Checklist Content']).tolist())
data = data[new_columns]
data=data.fillna(0)
cnt=0
cols_to_order = []
for i in data.columns:
    d=pd.unique(data[i])
    if len(d)<11:
        cols_to_order.append(i)
        cnt+=1
print(cnt)    
cols_to_order.remove('NonCompliance')
new_columns = (data.columns.drop(cols_to_order).tolist())
data = data[new_columns]


data['Industry Code']=pd.Categorical(data['Industry Code'])

data=pd.get_dummies(data)
#reorder y column as last
cols_to_order = ['NonCompliance']
new_columns =  (data.columns.drop(cols_to_order).tolist())+cols_to_order 
data = data[new_columns]
data=data.clip(lower=0)

dataset=data.to_numpy()
print(data)

264
       Business Age  Number of Employees  Total receivables  \
0                23                    7                0.0   
1               162                    6                0.0   
2               163                    8                0.0   
3                16                   10                0.0   
4                81                   46                0.0   
...             ...                  ...                ...   
63629           186                   29          6986448.0   
63630           203                   14          8589037.0   
63631           132                   15          1776367.0   
63632            33                    6           654569.0   
63633           130                    7          1928334.0   

       Trade receivables  Bank deposits, cash and the like  Company capital  \
0                    0.0                               0.0              0.0   
1                    0.0                               0.0              0.0   
2 

# Hyper parameter tuning

In [6]:
#HPT for MLP
y=data["NonCompliance"]
x=data.iloc[:, :-1]
x = SelectPercentile(chi2, percentile=50).fit_transform(x, y)

mlp_gs = MLPClassifier(max_iter=100)
parameter_space = {
        'hidden_layer_sizes': [(10,),(20,),(50,),(100,)],
        'activation': ['relu','logistic'],
        'alpha': [0.0001, 0.05],
        'learning_rate': ['constant', 'adaptive']
    }
start_time = time.time()
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=2, cv=4)
print(clf)

clf.fit(x,y)
traintime=(time.time()-start_time)
print("Training time: "+ str(traintime))
print(clf.best_params_)
print(clf.best_estimator_)
fh2=open("Log_" + "NNNonComplianceBestParameters"+ '.txt', 'w+')
fh2.write("Training time: "+ str(traintime))
fh2.write("Best parameters:"+ str(clf.best_params_))
fh2.write("Best estimator:"+ str(clf.best_estimator_))
fh2.close()

GridSearchCV(cv=4, estimator=MLPClassifier(max_iter=100), n_jobs=2,
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.0001, 0.05],
                         'hidden_layer_sizes': [(10,), (20,), (50,), (100,)],
                         'learning_rate': ['constant', 'adaptive']})
Training time: 683.8641488552094
{'activation': 'logistic', 'alpha': 0.0001, 'hidden_layer_sizes': (20,), 'learning_rate': 'adaptive'}
MLPClassifier(activation='logistic', hidden_layer_sizes=(20,),
              learning_rate='adaptive', max_iter=100)


In [7]:
#HPT for ADABoost
y=data["NonCompliance"]
x=data.iloc[:, :-1]
x = SelectPercentile(chi2, percentile=50).fit_transform(x, y)
mlp_gs = AdaBoostClassifier()
parameter_space = {
        'n_estimators': [10,20,50,100,200,500,1000],
        'learning_rate': [0.1,0.5,1.0,1.5]
    }
start_time = time.time()
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=8, cv=4)
print(clf)
    #clf.fit(X, y)
clf.fit(x,y)
traintime=(time.time()-start_time)
print("Training time: "+ str(traintime))
print(clf.best_params_)
print(clf.best_estimator_)
fh2=open("Log_" + "AdaboostNonComplianceBestParameters"+ '.txt', 'w+')
fh2.write("Training time: "+ str(traintime))
fh2.write("Best parameters:"+ str(clf.best_params_))
fh2.write("Best estimator:"+ str(clf.best_estimator_))
fh2.close()

GridSearchCV(cv=4, estimator=AdaBoostClassifier(), n_jobs=8,
             param_grid={'learning_rate': [0.1, 0.5, 1.0, 1.5],
                         'n_estimators': [10, 20, 50, 100, 200, 500, 1000]})
Training time: 3393.218388557434
{'learning_rate': 1.0, 'n_estimators': 200}
AdaBoostClassifier(n_estimators=200)


In [11]:
#HPT for GradBoost
y=data["NonCompliance"]
x=data.iloc[:, :-1]
x = SelectPercentile(chi2, percentile=50).fit_transform(x, y)
mlp_gs = GradientBoostingClassifier()
parameter_space = {
        'n_estimators': [10,20,50,100,200,500,1000],
        'learning_rate': [0.1,0.5,1.0,1.5]
    }
start_time = time.time()
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=8, cv=4)
print(clf)
    #clf.fit(X, y)
clf.fit(x,y)
traintime=(time.time()-start_time)
print("Training time: "+ str(traintime))
print(clf.best_params_)
print(clf.best_estimator_)
fh2=open("Log_" + "GradientNonComplianceBestParameters"+ '.txt', 'w+')
fh2.write("Training time: "+ str(traintime))
fh2.write("Best parameters:"+ str(clf.best_params_))
fh2.write("Best estimator:"+ str(clf.best_estimator_))
fh2.close()

GridSearchCV(cv=4, estimator=GradientBoostingClassifier(), n_jobs=8,
             param_grid={'learning_rate': [0.1, 0.5, 1.0, 1.5],
                         'n_estimators': [10, 20, 50, 100, 200, 500, 1000]})
Training time: 7436.930764913559
{'learning_rate': 0.1, 'n_estimators': 500}
GradientBoostingClassifier(n_estimators=500)


In [10]:
#HPT for k-NN
y=data["NonCompliance"]
x=data.iloc[:, :-1]
x = SelectPercentile(chi2, percentile=50).fit_transform(x, y)
mlp_gs = KNeighborsClassifier()
parameter_space = {
        'n_neighbors': [1,5,10,20,50,100,200,500,1000],
        'weights': ['uniform','distance']
    }
start_time = time.time()
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=8, cv=4)
print(clf)
    #clf.fit(X, y)
clf.fit(x,y)
traintime=(time.time()-start_time)
print("Training time: "+ str(traintime))
print(clf.best_params_)
print(clf.best_estimator_)
fh2=open("Log_" + "KNNNonComplianceBestParameters"+ '.txt', 'w+')
fh2.write("Training time: "+ str(traintime))
fh2.write("Best parameters:"+ str(clf.best_params_))
fh2.write("Best estimator:"+ str(clf.best_estimator_))
fh2.close()

GridSearchCV(cv=4, estimator=KNeighborsClassifier(), n_jobs=8,
             param_grid={'n_neighbors': [1, 5, 10, 20, 50, 100, 200, 500, 1000],
                         'weights': ['uniform', 'distance']})
Training time: 338.97323298454285
{'n_neighbors': 500, 'weights': 'uniform'}
KNeighborsClassifier(n_neighbors=500)


# Non-compliance experiment with feature selection and median based prediction thresholds

In [17]:
#override ML methods to create holdout sets for training the models and to set prediction thresholds based on prediction medians from the holdout sets.
class MyMLPClassifier(MLPClassifier):
    thold=0
    def fit(self,X,y,**kwargs):
        #Train data consist of 80% of the data set. The holdout is 10% of the training data - also 8% in total.
        holdout=X[round(0.9*len(X[:,0]))+1:len(X[:,0]),:]
        train=X[0:round(0.9*len(X[:,0])),:]
        y_train=y[0:round(0.9*len(X[:,0]))]
        super(MyMLPClassifier, self).fit(train,y_train)
        result=super(MyMLPClassifier, self).predict_proba(holdout, **kwargs)
        #Using prediction median from the holdout set to create thresholds.
        self.thold=np.median(result[:,1])
    def predict(self,X, threshold=0.5, **kwargs):
        result = super(MyMLPClassifier, self).predict_proba(X, **kwargs)
        predictions=[1 if p>self.thold else 0 for p in result[:,1]]
        return predictions
    
class MyLogisticRegressionClassifier(LogisticRegression):
    thold=0
    def fit(self,X,y,**kwargs):
        holdout=X[round(0.9*len(X[:,0]))+1:len(X[:,0]),:]
        train=X[0:round(0.9*len(X[:,0])),:]
        y_train=y[0:round(0.9*len(X[:,0]))]
        super(MyLogisticRegressionClassifier, self).fit(train,y_train)
        result=super(MyLogisticRegressionClassifier, self).predict_proba(holdout, **kwargs)
        self.thold=np.median(result[:,1])
    def predict(self,X, threshold=0.5, **kwargs):
        result = super(MyLogisticRegressionClassifier, self).predict_proba(X, **kwargs)
        predictions=[1 if p>self.thold else 0 for p in result[:,1]]
        return predictions
class MyGaussianNBClassifier(GaussianNB):
    thold=0
    def fit(self,X,y,**kwargs):
        holdout=X[round(0.9*len(X[:,0]))+1:len(X[:,0]),:]
        train=X[0:round(0.9*len(X[:,0])),:]
        y_train=y[0:round(0.9*len(X[:,0]))]
        super(MyGaussianNBClassifier, self).fit(train,y_train)
        result=super(MyGaussianNBClassifier, self).predict_proba(holdout, **kwargs)
        self.thold=np.median(result[:,1])
    def predict(self,X, threshold=0.5, **kwargs):
        result = super(MyGaussianNBClassifier, self).predict_proba(X, **kwargs)
        predictions=[1 if p>self.thold else 0 for p in result[:,1]]
        return predictions
class MyDecisionTreeClassifier(DecisionTreeClassifier):
    thold=0
    def fit(self,X,y,**kwargs):
        holdout=X[round(0.9*len(X[:,0]))+1:len(X[:,0]),:]
        train=X[0:round(0.9*len(X[:,0])),:]
        y_train=y[0:round(0.9*len(X[:,0]))]
        super(MyDecisionTreeClassifier, self).fit(train,y_train)
        result=super(MyDecisionTreeClassifier, self).predict_proba(holdout, **kwargs)
        self.thold=np.median(result[:,1])
    def predict(self,X, threshold=0.5, **kwargs):
        result = super(MyDecisionTreeClassifier, self).predict_proba(X, **kwargs)
        predictions=[1 if p>self.thold else 0 for p in result[:,1]]
        return predictions
class MyKNeighborsClassifier(KNeighborsClassifier):
    thold=0
    def fit(self,X,y,**kwargs):
        holdout=X[round(0.9*len(X[:,0]))+1:len(X[:,0]),:]
        train=X[0:round(0.9*len(X[:,0])),:]
        y_train=y[0:round(0.9*len(X[:,0]))]
        super(MyKNeighborsClassifier, self).fit(train,y_train)
        result=super(MyKNeighborsClassifier, self).predict_proba(holdout, **kwargs)
        self.thold=np.median(result[:,1])
    def predict(self,X, threshold=0.5, **kwargs):
        result = super(MyKNeighborsClassifier, self).predict_proba(X, **kwargs)
        predictions=[1 if p>self.thold else 0 for p in result[:,1]]
        return predictions
class MyAdaBoostClassifier(AdaBoostClassifier):
    thold=0
    def fit(self,X,y,**kwargs):
        holdout=X[round(0.9*len(X[:,0]))+1:len(X[:,0]),:]
        train=X[0:round(0.9*len(X[:,0])),:]
        y_train=y[0:round(0.9*len(X[:,0]))]
        super(MyAdaBoostClassifier, self).fit(train,y_train)
        result=super(MyAdaBoostClassifier, self).predict_proba(holdout, **kwargs)
        self.thold=np.median(result[:,1])
    def predict(self,X, threshold=0.5, **kwargs):
        result = super(MyAdaBoostClassifier, self).predict_proba(X, **kwargs)
        predictions=[1 if p>self.thold else 0 for p in result[:,1]]
        return predictions
class MyGradientBoostingClassifier(GradientBoostingClassifier):
    thold=0
    def fit(self,X,y,**kwargs):
        holdout=X[round(0.9*len(X[:,0]))+1:len(X[:,0]),:]
        train=X[0:round(0.9*len(X[:,0])),:]
        y_train=y[0:round(0.9*len(X[:,0]))]
        super(MyGradientBoostingClassifier, self).fit(train,y_train)
        result=super(MyGradientBoostingClassifier, self).predict_proba(holdout, **kwargs)
        self.thold=np.median(result[:,1])
    def predict(self,X, threshold=0.5, **kwargs):
        result = super(MyGradientBoostingClassifier, self).predict_proba(X, **kwargs)
        predictions=[1 if p>self.thold else 0 for p in result[:,1]]
        return predictions
    
X = dataset[:, :-1]
y = dataset[:, -1]

X_new=0
#Iterate through all ML methods
for j in [MyMLPClassifier(alpha= 0.0001,activation='logistic', hidden_layer_sizes=(20,),
              learning_rate='adaptive', max_iter=100),MyLogisticRegressionClassifier(),MyGaussianNBClassifier(),MyDecisionTreeClassifier(),MyKNeighborsClassifier(n_neighbors=500, weights='uniform'),MyAdaBoostClassifier(learning_rate=0.5, n_estimators=200),MyGradientBoostingClassifier(learning_rate=0.1,n_estimators=500)]:
    timetotal1=0
    xscore_chi_bal_acc=0
    xscore_chi_bal_acc_std=0
    xscore_chi_acc=0
    xscore_chi_acc_std=0
    xscore_chi_prec=0
    xscore_chi_prec_std=0
    xscore_chi_rec=0
    xscore_chi_rec_std=0
    xscore_chi_roc_auc=0
    xscore_chi_roc_auc_std=0
    xscore_f_bal_acc=0
    xscore_f_bal_acc_std=0
    xscore_f_acc=0
    xscore_f_acc_std=0
    xscore_f_prec=0
    xscore_f_prec_std=0
    xscore_f_rec=0
    xscore_f_rec_std=0
    xscore_f_roc_auc=0
    xscore_f_roc_auc_std=0
    timetotal2=0
    xscore2=0
    #Iterate through all feature selection set sizes
    for i in [0.005,0.01,0.05,0.1,0.2,0.3,0.4,0.5]: #Feature selection, numbers are percentages of the features that will be selected
        model=j
        #Perform feature selection with chi2
        X_new = SelectPercentile(chi2, percentile=i*100).fit_transform(X, y)
        start_time1=time.time()
        #Perform crossvalidation
        scores=cross_validate(model, X_new, y, cv=5,scoring=['accuracy','balanced_accuracy','precision','recall','roc_auc'])
        #print(scores)
        if i==0.005: #Simple mechanism for initializing the scores.
            xscore_chi_bal_acc=mean(scores['test_balanced_accuracy'])
            xscore_chi_bal_acc_std=statistics.stdev(scores['test_balanced_accuracy'])
            xscore_chi_acc=mean(scores['test_accuracy'])
            xscore_chi_acc_std=statistics.stdev(scores['test_accuracy'])
            xscore_chi_prec=mean(scores['test_precision'])
            xscore_chi_prec_std=statistics.stdev(scores['test_precision'])
            xscore_chi_rec=mean(scores['test_recall'])
            xscore_chi_rec_std=statistics.stdev(scores['test_recall'])
            xscore_chi_roc_auc=mean(scores['test_roc_auc'])
            xscore_chi_roc_auc_std=statistics.stdev(scores['test_roc_auc'])
        else:
            xscore_chi_bal_acc+=mean(scores['test_balanced_accuracy'])
            xscore_chi_bal_acc_std+=statistics.stdev(scores['test_balanced_accuracy'])
            xscore_chi_acc+=mean(scores['test_accuracy'])
            xscore_chi_acc_std+=statistics.stdev(scores['test_accuracy'])
            xscore_chi_prec+=mean(scores['test_precision'])
            xscore_chi_prec_std+=statistics.stdev(scores['test_precision'])
            xscore_chi_rec+=mean(scores['test_recall'])
            xscore_chi_rec_std+=statistics.stdev(scores['test_recall'])
            xscore_chi_roc_auc+=mean(scores['test_roc_auc'])
            xscore_chi_roc_auc_std+=statistics.stdev(scores['test_roc_auc'])
        timetotal1+=(time.time()-start_time1)
        
        #Perform feature selection with anova f
        X_new = SelectPercentile(f_classif, percentile=i*100).fit_transform(X, y)
        start_time2=time.time()
        #Perform crossvalidation
        scores=cross_validate(model, X_new, y, cv=5,scoring=['accuracy','balanced_accuracy','precision','recall','roc_auc'])
        if i==0.005:
            xscore_f_bal_acc=mean(scores['test_balanced_accuracy'])
            xscore_f_bal_acc_std=statistics.stdev(scores['test_balanced_accuracy'])
            xscore_f_acc=mean(scores['test_accuracy'])
            xscore_f_acc_std=statistics.stdev(scores['test_accuracy'])
            xscore_f_prec=mean(scores['test_precision'])
            xscore_f_prec_std=statistics.stdev(scores['test_precision'])
            xscore_f_rec=mean(scores['test_recall'])
            xscore_f_rec_std=statistics.stdev(scores['test_recall'])
            xscore_f_roc_auc=mean(scores['test_roc_auc'])
            xscore_f_roc_auc_std=statistics.stdev(scores['test_roc_auc'])
        else:
            xscore_f_bal_acc+=mean(scores['test_balanced_accuracy'])
            xscore_f_bal_acc_std+=statistics.stdev(scores['test_balanced_accuracy'])
            xscore_f_acc+=mean(scores['test_accuracy'])
            xscore_f_acc_std+=statistics.stdev(scores['test_accuracy'])
            xscore_f_prec+=mean(scores['test_precision'])
            xscore_f_prec_std+=statistics.stdev(scores['test_precision'])
            xscore_f_rec+=mean(scores['test_recall'])
            xscore_f_rec_std+=statistics.stdev(scores['test_recall'])
            xscore_f_roc_auc+=mean(scores['test_roc_auc'])
            xscore_f_roc_auc_std+=statistics.stdev(scores['test_roc_auc'])
        timetotal2+=(time.time()-start_time2)
#Calculate average statistics over all feature selection set sizes
    print(str(j)+" Avg time: "+str((timetotal1+timetotal2)/16))
    print(str(j)+" chi2 bal. acc: "+str(xscore_chi_bal_acc/8)+"+-"+str(xscore_chi_bal_acc_std/8)+" acc: "+str(xscore_chi_acc/8)+"+-"+str(xscore_chi_acc_std/8)+" prec: "+str(xscore_chi_prec/8)+"+-"+str(xscore_chi_prec_std/8)+" rec: "+str(xscore_chi_rec/8)+"+-"+str(xscore_chi_rec_std/8)+" auc: "+str(xscore_chi_roc_auc/8)+"+-"+str(xscore_chi_roc_auc_std/8))
    print(str(j)+" Anova F bal. acc: "+str(xscore_f_bal_acc/8)+"+-"+str(xscore_f_bal_acc_std/8)+" acc: "+str(xscore_f_acc/8)+"+-"+str(xscore_f_acc_std/8)+" prec: "+str(xscore_f_prec/8)+"+-"+str(xscore_f_prec_std/8)+" rec: "+str(xscore_f_rec/8)+"+-"+str(xscore_f_rec_std/8)+" auc: "+str(xscore_f_roc_auc/8)+"+-"+str(xscore_f_roc_auc_std/8))
   

MyMLPClassifier(activation='logistic', hidden_layer_sizes=(20,),
                learning_rate='adaptive', max_iter=100) Avg time: 24.467836260795593
MyMLPClassifier(activation='logistic', hidden_layer_sizes=(20,),
                learning_rate='adaptive', max_iter=100) chi2 bal. acc: 0.5254791666663298+-0.009784855537240145 acc: 0.40367960166185773+-0.02572005487840897 prec: 0.7843924437820339+-0.018881389311373325 rec: 0.2746191685179308+-0.05338199726138709 auc: 0.5280167533621987+-0.012707672261608098
MyMLPClassifier(activation='logistic', hidden_layer_sizes=(20,),
                learning_rate='adaptive', max_iter=100) Anova F bal. acc: 0.5385606381825213+-0.006465350717052809 acc: 0.38963806848026444+-0.021921399456004312 prec: 0.8212765561895918+-0.022473457536763053 rec: 0.23183909869882577+-0.045590494760379505 auc: 0.5594723194703138+-0.01576834775956161
MyLogisticRegressionClassifier() Avg time: 3.29045207798481
MyLogisticRegressionClassifier() chi2 bal. acc: 0.4215047329291

# Top Features

In [18]:
X = dataset[:, :-1]
y = dataset[:, -1]
X_new = SelectPercentile(chi2, percentile=0.05*100)

train_dataframe = data.drop("NonCompliance", axis=1)
x_features = X_new.fit_transform(X, y)
columns = np.asarray(train_dataframe.columns.values)
support = np.asarray(X_new.get_support())
columns_with_support = columns[support]

print("Chi2 top 5% selected features")
print(columns_with_support)
X_new = SelectPercentile(f_classif, percentile=0.05*100)

train_dataframe = data.drop("NonCompliance", axis=1)
x_features = X_new.fit_transform(X, y)
columns = np.asarray(train_dataframe.columns.values)
support = np.asarray(X_new.get_support())
columns_with_support = columns[support]
print("Anova F top 5% selected features")
print(columns_with_support)



Chi2 top 5% selected features
['Total receivables' 'Trade receivables' 'Other operating cost'
 'TOTAL ASSETS' 'Machinery and plant' 'Total debt'
 'Total net loans and receivables from credit institutions' 'Bond debt'
 'Total liabilities to credit institutions' 'Issued by the public'
 'Other current liabilities' 'Investment in subsidiaries'
 'Repayment loans' 'Total other liabilities' 'Payroll cost'
 'Accounts payable' 'Total invested equity' 'Total current assets'
 'Total fixed assets' 'Total fixed assets.1' 'Total current liabilities'
 'Total equity' 'Total revenue' 'Sales revenue'
 'TOTAL LIABILITIES AND EQUITY'
 'Total liabilities established by issuance of securities'
 'Cash and receivables from central banks' 'Contingent liabilities'
 'TOTAL EQUITY AND LIABILITIES.1' 'Total costs' 'Cost of goods'
 'Guarantees' 'Total of financial fixed assets'
 'Total deposits from and liabilities to customers' 'Other debt'
 'Total net loans and receivables on customers'
 'Loans to and receivables